In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import urllib.request
from PIL import Image

In [2]:
#Read csv
columnas  = pd.read_csv('columns_meaning.csv', index_col=0)
columnas

,Description
Column,
0,Case_ID registered in Hospital
1,Unique code for the Hospital
2,Unique code for the type of Hospital
3,City Code of the Hospital
4,Region Code of the Hospital
5,Number of Extra rooms available in the Hospital
6,Department overlooking the case
7,Code for the Ward type
8,Code for the Ward Facility


In [3]:
column_list = list(columnas['Description'])
column_list

['Case_ID registered in Hospital',
 'Unique code for the Hospital',
 'Unique code for the type of Hospital',
 'City Code of the Hospital',
 'Region Code of the Hospital',
 'Number of Extra rooms available in the Hospital',
 'Department overlooking the case',
 'Code for the Ward type',
 'Code for the Ward Facility',
 'Condition of Bed in the Ward',
 'Unique Patient Id',
 'City Code for the patient',
 'Admission Type registered by the Hospital',
 'Severity of the illness recorded at the time of admission',
 'Number of Visitors with the patient',
 'Age of the patient',
 'Deposit at the Admission Time',
 'Stay Days by the patient']

In [4]:
train = pd.read_csv('hospital_train.csv')
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
1,159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
2,309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
3,279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
4,147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
99996,254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
99997,69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40
99998,204442,32,f,9,Y,2,gynecology,S,B,4.0,113798,15.0,Trauma,Moderate,3,41-50,5092.0,11-20


In [5]:
#Incorporamos los nombres de las columnas para mejor comprensión
train.columns = column_list
train

,Case_ID registered in Hospital,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time,Stay Days by the patient
0,161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
1,159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
2,309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
3,279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
4,147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
99996,254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
99997,69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40
99998,204442,32,f,9,Y,2,gynecology,S,B,4.0,113798,15.0,Trauma,Moderate,3,41-50,5092.0,11-20


In [6]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                                                     Non-Null Count   Dtype  
---  ------                                                     --------------   -----  
 0   Case_ID registered in Hospital                             100000 non-null  int64  
 1   Unique code for the Hospital                               100000 non-null  int64  
 2   Unique code for the type of Hospital                       100000 non-null  object 
 3   City Code of the Hospital                                  100000 non-null  int64  
 4   Region Code of the Hospital                                100000 non-null  object 
 5   Number of Extra rooms available in the Hospital            100000 non-null  int64  
 6   Department overlooking the case                            100000 non-null  object 
 7   Code for the Ward type                                     100000 non-null  object 


In [7]:
train.isnull().sum()

Case_ID registered in Hospital                                  0
Unique code for the Hospital                                    0
Unique code for the type of Hospital                            0
City Code of the Hospital                                       0
Region Code of the Hospital                                     0
Number of Extra rooms available in the Hospital                 0
Department overlooking the case                                 0
Code for the Ward type                                          0
Code for the Ward Facility                                      0
Condition of Bed in the Ward                                   33
Unique Patient Id                                               0
City Code for the patient                                    1483
Admission Type registered by the Hospital                       0
Severity of the illness recorded at the time of admission       0
Number of Visitors with the patient                             0
Age of the

In [8]:
train['Condition of Bed in the Ward'].value_counts()


2.0    38932
3.0    34548
4.0    18096
1.0     8391
Name: Condition of Bed in the Ward, dtype: int64

In [9]:
#Seleccionamos las columnas más relevantes por intuición sobre la estancia de un paciente, como el número de habitaciones que hay disponibles, el deposito economico realizado, severidad de la enfermedad, la edad o si existen muchos visitantes puesto que puede influuir en el numero de personas que pueden estar a su cuidado de forma externa y en la motivación del paciente.
train_1 = train[['City Code of the Hospital','Number of Extra rooms available in the Hospital','Condition of Bed in the Ward','Number of Visitors with the patient','Deposit at the Admission Time','Department overlooking the case', 'Severity of the illness recorded at the time of admission', 'Age of the patient']]
train_1

,City Code of the Hospital,Number of Extra rooms available in the Hospital,Condition of Bed in the Ward,Number of Visitors with the patient,Deposit at the Admission Time,Department overlooking the case,Severity of the illness recorded at the time of admission,Age of the patient
0,6,2,4.0,2,2817.0,gynecology,Moderate,21-30
1,6,4,2.0,4,4498.0,gynecology,Moderate,51-60
2,5,2,3.0,2,4573.0,anesthesia,Moderate,71-80
3,9,3,4.0,4,7202.0,gynecology,Moderate,11-20
4,1,3,2.0,2,3398.0,gynecology,Moderate,51-60
...,...,...,...,...,...,...,...,...
99995,9,3,3.0,6,3966.0,gynecology,Moderate,51-60
99996,11,2,2.0,3,4005.0,gynecology,Moderate,21-30
99997,6,3,3.0,2,5215.0,gynecology,Minor,31-40
99998,9,2,4.0,3,5092.0,gynecology,Moderate,41-50


In [10]:
#Tras intentar eliminar los valores NaN en la columna Condition of Bed in the Ward sin dar resultado, se cambian por un valor que no interfiera con el resto de categorías
train_1 = train_1.fillna(0)

In [11]:
def codi(df):
    '''
    Esta función da valores numéricos a las categorías de las columnas object relevantes
    '''
    cols = ['Department overlooking the case', 'Severity of the illness recorded at the time of admission', 'Age of the patient']
    for col in cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df


In [12]:
codi(train_1)

,City Code of the Hospital,Number of Extra rooms available in the Hospital,Condition of Bed in the Ward,Number of Visitors with the patient,Deposit at the Admission Time,Department overlooking the case,Severity of the illness recorded at the time of admission,Age of the patient
0,6,2,4.0,2,2817.0,2,2,2
1,6,4,2.0,4,4498.0,2,2,5
2,5,2,3.0,2,4573.0,1,2,7
3,9,3,4.0,4,7202.0,2,2,1
4,1,3,2.0,2,3398.0,2,2,5
...,...,...,...,...,...,...,...,...
99995,9,3,3.0,6,3966.0,2,2,5
99996,11,2,2.0,3,4005.0,2,2,2
99997,6,3,3.0,2,5215.0,2,1,3
99998,9,2,4.0,3,5092.0,2,2,4


In [13]:
train_1.isnull().sum()

City Code of the Hospital                                    0
Number of Extra rooms available in the Hospital              0
Condition of Bed in the Ward                                 0
Number of Visitors with the patient                          0
Deposit at the Admission Time                                0
Department overlooking the case                              0
Severity of the illness recorded at the time of admission    0
Age of the patient                                           0
dtype: int64

In [14]:
#En el modelo 2 cambiamos el tamaño del train
X = np.array(train_1)
y = np.array(train['Stay Days by the patient'])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=153)

reg_log = Pipeline(steps=[
    ('classifier', SVC())])

logistic_params = {'classifier': [LogisticRegression()]}
kn_params = {'classifier': [KNeighborsClassifier()]}
svm_params = {'classifier': [SVC()]}

search_space = [
    logistic_params,
    kn_params,
    svm_params]

clf = GridSearchCV(estimator = reg_log,
                param_grid = search_space,
                cv = 2,
                verbose=1,
                n_jobs=-1)



In [15]:
clf.fit(X_train, y_train)


Fitting 2 folds for each of 3 candidates, totalling 6 fits
C:\Users\marin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=2, estimator=Pipeline(steps=[('classifier', SVC())]), n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression()]},
                         {'classifier': [KNeighborsClassifier()]},
                         {'classifier': [SVC()]}],
             verbose=1)

In [16]:
print(clf.best_params_)
print(clf.best_score_)

{'classifier': LogisticRegression()}
0.3327625


In [17]:
predictions = clf.predict(X_test)
print(predictions)

['21-30' '21-30' '11-20' ... '21-30' '21-30' '21-30']


In [18]:
test = pd.read_csv('hospital_test.csv')
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
1,208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
2,305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
3,266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
4,13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133739,318155,18,d,13,Y,6,anesthesia,Q,B,3.0,123269,25.0,Urgent,Minor,4,51-60,5089.0
133740,144850,27,a,7,Y,2,gynecology,S,C,2.0,1293,8.0,Emergency,Moderate,3,61-70,6713.0
133741,180676,16,c,3,Z,3,gynecology,R,A,3.0,112921,5.0,Trauma,Minor,3,31-40,5326.0
133742,39933,28,b,11,X,4,gynecology,R,F,2.0,585,2.0,Trauma,Extreme,2,31-40,7072.0


In [19]:
column_test = column_list[0:-1]

In [20]:
test.columns = column_test
test

,Case_ID registered in Hospital,Unique code for the Hospital,Unique code for the type of Hospital,City Code of the Hospital,Region Code of the Hospital,Number of Extra rooms available in the Hospital,Department overlooking the case,Code for the Ward type,Code for the Ward Facility,Condition of Bed in the Ward,Unique Patient Id,City Code for the patient,Admission Type registered by the Hospital,Severity of the illness recorded at the time of admission,Number of Visitors with the patient,Age of the patient,Deposit at the Admission Time
0,314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
1,208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
2,305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
3,266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
4,13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133739,318155,18,d,13,Y,6,anesthesia,Q,B,3.0,123269,25.0,Urgent,Minor,4,51-60,5089.0
133740,144850,27,a,7,Y,2,gynecology,S,C,2.0,1293,8.0,Emergency,Moderate,3,61-70,6713.0
133741,180676,16,c,3,Z,3,gynecology,R,A,3.0,112921,5.0,Trauma,Minor,3,31-40,5326.0
133742,39933,28,b,11,X,4,gynecology,R,F,2.0,585,2.0,Trauma,Extreme,2,31-40,7072.0


In [21]:
test_1 = test[['City Code of the Hospital','Number of Extra rooms available in the Hospital','Condition of Bed in the Ward','Number of Visitors with the patient','Deposit at the Admission Time','Department overlooking the case', 'Severity of the illness recorded at the time of admission', 'Age of the patient']]

In [22]:
test_1 = test_1.fillna(0)

In [23]:
codi(test_1)

,City Code of the Hospital,Number of Extra rooms available in the Hospital,Condition of Bed in the Ward,Number of Visitors with the patient,Deposit at the Admission Time,Department overlooking the case,Severity of the illness recorded at the time of admission,Age of the patient
0,7,4,2.0,2,4778.0,2,1,4
1,5,3,2.0,3,5734.0,2,2,3
2,1,4,4.0,3,5064.0,2,1,7
3,3,4,2.0,4,3254.0,0,0,3
4,6,4,1.0,3,4639.0,2,1,2
...,...,...,...,...,...,...,...,...
133739,13,6,3.0,4,5089.0,1,1,5
133740,7,2,2.0,3,6713.0,2,2,6
133741,3,3,3.0,3,5326.0,2,1,3
133742,11,4,2.0,2,7072.0,2,0,3


In [24]:
X_pred = np.array(test_1)

In [25]:
X_pred.shape

(133744, 8)

In [26]:
predictions_submit = clf.predict(X_pred)
predictions_submit

array(['21-30', '21-30', '21-30', ..., '11-20', '21-30', '21-30'],
      dtype=object)

In [27]:
#Reading sample to compare
sample = pd.read_csv('sample_submission.csv')
sample

,id,days
0,314114,11-20
1,208989,31-40
2,305872,81-90
3,266099,21-30
4,13228,31-40
...,...,...
133739,318155,41-50
133740,144850,21-30
133741,180676,11-20
133742,39933,21-30


In [28]:
submission = pd.DataFrame({"id": sample['id'], "days": predictions_submit})
submission

,id,days
0,314114,21-30
1,208989,21-30
2,305872,21-30
3,266099,31-40
4,13228,21-30
...,...,...
133739,318155,21-30
133740,144850,21-30
133741,180676,11-20
133742,39933,21-30


In [29]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission3.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")

In [30]:
chequeator(submission)

You're ready to submit!
